In [1]:
# only for development
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from diffractem import io, tools
from diffractem.stream_parser import StreamParser, make_substream
import numpy as np
import pandas as pd
import os
import matplotlib
import seaborn as sns
bin_path = '/opts/crystfel_latest/bin/' # might be different than standard
from glob import glob
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

# Merging and anaysis of dose-fractionated sets
...works essentially just as in `merging.ipynb`, just additionally making use of `partialator`s `custom-split` capability to make sub-sets with the individual frame numbers.

To make this work, we first need to get a `hits_allframe_split.txt` file which contains labels for the custom split, that is, the frame number per crystal, to which each Event in the file belongs.

In [3]:
stream = StreamParser('streams/hits_allframe.stream')
print(f'Stream file contains {stream.num_crystals} crystals in {stream.num_shots} shots')
split_data = stream.shots[['file', 'Event', 'header/int//%/shots/frame']]
split_data.to_csv(stream.filename.rsplit('.', 1)[0] + '_split.txt', index=False, header=False, sep=' ')

!head -n 11 streams/hits_allframe_split.txt # have a look

Stream file contains 12750 crystals in 13220 shots
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//0 0
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//1 1
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//2 2
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//3 3
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//4 4
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//5 5
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//6 6
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//7 7
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//8 8
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//9 9
proc_data/LysoS1_001_00000_allframe_hit.h5 entry//10 0


## Preparation of a partialator script
...just as in `merging.ipynb`, just that we do not scan the `stop-after`, but instead set `split=True`.

In [4]:
# get a list of stream files
stream_list = glob('streams/hits_allframe.stream')

popts = {'no-polarisation': True, 'no-Bscale': False, 'no-scale': False, 
        'force-bandwidth': 2e-5,  'force-radius': False, 'force-lambda': 0.0251,
            'push-res': 1.4,  'min-measurements': [3, ], 'model': ['unity', 'xsphere'],
            'symmetry': '422', 'no-logs': False, 'iterations': 3, 'j': 10}

# you need to set those if you want to use slurm to submit merging runs
slurm_opts = {'C': 'scratch', 
                'partition': 'medium', 
                'time': '"04:00:00"',
                'nodes': 1}

settings = tools.call_partialator(stream_list, popts, par_runs=4, 
                       split=True, out_dir='merged_frac',
                       slurm=False, cache_streams=False, 
                       slurm_opts=slurm_opts)

!chmod +x partialator_run.sh

Please run partialator_run.sh to start merging.


In [ ]:
# example how to send data to a cluster
# !scp -r streams rbuecke1@transfer.gwdg.de:~/SHARED/EDIFF/temp
!scp partialator_run.sh rbuecke1@transfer.gwdg.de:~/SHARED/EDIFF/temp

In [ ]:
# example how to get it back from a cluster
# %mkdir merged
# !scp 'rbuecke1@transfer.gwdg.de:~/SHARED/EDIFF/temp/merged/*.hkl*' merged/

## Analyze and validate results
...again, the only difference to `merging.ipynb` being that `custom_split=True` in the first command.

In [5]:
# check what hkls we have available....
settings = tools.get_hkl_settings('merged_frac/hits_allframe*-*.hkl', unique_only=True, custom_split=True)

if 'input' in settings.columns:
    settings['input'] = settings['input'].str.rsplit('/', 1, expand=True).iloc[:,-1]

default_symmetry = '422'
highres = 1.75 # highest shell, in A
nshells = 10

# tools.analyze_hkl() #...is used using ProcessPoolExecutor

ftrs = {}
with ProcessPoolExecutor() as exc:
    for _, s in settings.iterrows():
        ftrs[s.hklfile] = exc.submit(tools.analyze_hkl, fn=s.hklfile, cell='refined.cell', 
                              point_group=s.symmetry if 'symmetry' in s else default_symmetry, 
                              highres=highres, nshells=nshells, bin_path='/opts/crystfel_master/bin')    

err = {lbl: v.exception() for lbl, v in ftrs.items() if v.exception()}
if err:
    print('Analysis gave errors!', str(err))
out = {lbl: v.result() for lbl, v in ftrs.items() if not v.exception()}

sd = pd.concat([v.result()[0].assign(hklfile=lbl) 
                for lbl, v in ftrs.items() 
                    if not v.exception()], axis=0).merge(
    settings, on='hklfile')

overall = pd.concat([pd.DataFrame(v.result()[1], index=[lbl])
                for lbl, v in ftrs.items() 
                     if not v.exception()], axis=0).merge(
    settings, right_on='hklfile', left_index=True).rename(
    columns={'<snr>': 'SNR', 'redundancy': 'Red', 'completeness': 'Compl', 'CC*': 'CCstar'})

# write out results
%rm -f shell/*
for ident, grp in sd.groupby(['hklfile']):
    grp.sort_values('Center 1/nm')[['Center 1/nm', 'nref', 'Possible', 'Compl', 'Meas', 'Red', 'SNR',
           'Mean', 'd/A', 'Min 1/nm', 'Max 1/nm', 'CC', 'CCstar',
           'Rsplit']].to_csv(f'shell/{ident.rsplit("/",1)[-1]}.csv', index=False, float_format='%.2f')

#### Example to show results
...using the result DataFrame's `pivot` function.

In [6]:
# convenient function to get FOMs. Set the one you want as 'value'
sd.pivot(index='d/A', columns='hklfile', values=['CC']).sort_index(ascending=False)

CC  \
hklfile merged_frac/hits_allframe__3__unity-0.hkl   
d/A                                                 
6.80                                     0.894263   
3.34                                     0.863738   
2.79                                     0.727442   
2.49                                     0.569118   
2.29                                     0.540939   
2.14                                     0.373143   
2.02                                     0.310046   
1.93                                     0.252844   
1.85                                     0.182227   
1.78                                     0.036999   

                                                   \
hklfile merged_frac/hits_allframe__3__unity-1.hkl   
d/A                                                 
6.80                                     0.890570   
3.34                                     0.892459   
2.79                                     0.809905   
2.49                                     0.682201   
2.29                                     0.675665   
2.14                                     0.491047   
2.02                                     0.448413   
1.93                                     0.371027   
1.85                                     0.212516   
1.78                                     0.005975   

                                                   \
hklfile merged_frac/hits_allframe__3__unity-2.hkl   
d/A                                                 
6.80                                     0.898200   
3.34                                     0.873871   
2.79                                     0.784169   
2.49                                     0.652567   
2.29                                     0.585916   
2.14                                     0.451207   
2.02                                     0.304287   
1.93                                     0.244866   
1.85                                     0.035432   
1.78                                     0.055575   

                                                   \
hklfile merged_frac/hits_allframe__3__unity-3.hkl   
d/A                                                 
6.80                                     0.881846   
3.34                                     0.862829   
2.79                                     0.744783   
2.49                                     0.534936   
2.29                                     0.505474   
2.14                                     0.326829   
2.02                                     0.215154   
1.93                                     0.124577   
1.85                                     0.126743   
1.78                                    -0.031129   

                                                   \
hklfile merged_frac/hits_allframe__3__unity-4.hkl   
d/A                                                 
6.80                                     0.904125   
3.34                                     0.877579   
2.79                                     0.680304   
2.49                                     0.395660   
2.29                                     0.383440   
2.14                                     0.225954   
2.02                                     0.110463   
1.93                                     0.059729   
1.85                                     0.007111   
1.78                                     0.084117   

                                                   \
hklfile merged_frac/hits_allframe__3__unity-5.hkl   
d/A                                                 
6.80                                     0.905594   
3.34                                     0.864587   
2.79                                     0.610310   
2.49                                     0.342205   
2.29                                     0.174699   
2.14                                     0.160447   
2.02                                     0.082001   
1.93                                     0.040893   
1.85          

## Analysis Plot
...as Fig. 1 in `merging.ipynb`.

In [8]:
# %matplotlib inline
%matplotlib widget

model = 'unity' # 'unity' or 'xsphere'

# SETTINGS ---

fh, axs = plt.subplots(2, 2, figsize=(18/2.54,15/2.54), dpi=120, sharex=True)
lsp, lrow = 0.85, 3 # space near top left for legend, and # of legend columns

# pick your FOMs and their y ranges
FOMs = [('CC', 0, 1), ('Mean', 0, 40), ('Compl', 0, 100), ('Red', 0, 100)]
sdsel = sd.query(f'model == "{model}"')
angstrom = False # if True, show x axis in A, instead of 1/nm

# ------

try:
    import seaborn as sns
    sns.set('notebook','whitegrid') # optional. Set a style...
except:
    print('Seaborn not installed, it seems.')

axs = axs.ravel()

# ids = get_id_table(sdsel['identifier'])
idcols = [cn for cn, col in sdsel[settings.columns].iteritems() 
          if len(col.unique()) > 1 and (cn != 'hklfile')]
print('Legend is', ' '.join(idcols))

for ident, grp in sdsel.groupby(['hklfile']):
    
    ls = '-'
     
    lbl = tuple(grp[idcols].drop_duplicates().values.astype(str).ravel())
    
    for ax, (fom, ymin, ymax) in zip(axs, FOMs):
        ax.plot(grp['d/A'] if angstrom else grp['Center 1/nm'], grp[fom], 
                label=' '.join(lbl), ls=ls)
        ax.set_title(fom)
        ax.set_ylim((ymin, ymax))
        if angstrom:
            ax.set_xlim(sorted(ax.get_xlim(), reverse=True))
        ax.grid(True)
        if fom in ['CC', 'CCstar']:
            ax.axhline(0.143 if fom == 'CC' else 0.5,ls=':')
        
lg = fh.legend(*ax.get_legend_handles_labels(), ncol=lrow, 
               fontsize='xx-small', loc='lower center', 
               bbox_to_anchor=(0.5, lsp), frameon=True)
axs[-1].set_xlabel(r'Resolution shell/Å' if angstrom else r'Resolution shell/nm$^{-1}$')
plt.draw()
# lpos = lg.get_window_extent()

fh.subplots_adjust(wspace=0.3, top=lsp-0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Legend is split_label
